# Requirements
1. Pandas
2. Seaborn
3. NLTK (vader)

Run the program sequentially

Importing Libraries


In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Dataset
The data is already preprocessed in excel

In [91]:
df=pd.read_csv('Satisfaction_S.csv')
df.head()

,SN,flight_number,origin_station_code,destination_station_code,record_locator,scheduled_departure_date,question_text,score,entrée_score,actual_score,...,equipment_type_code,ua_uax,actual_flown_miles,haul_type,departure_gate,arrival_gate,international_domestic_indicator,response_group,media_provider,hub_spoke
0,16621,2014,SFX,ORX,A0XX00,09-11-2022,How satisfied were you with the food & beverag...,5,NaN,5,...,77M,UA,1846,Medium,G2,C16,Domestic,member,PANASONIC,hub departure
1,20635,5308,SMX,SFX,A0XX08,09-14-22,How satisfied were you with the food & beverag...,5,NaN,5,...,CR7,UAX,86,Short,A14,E3,Domestic,member,GOGO,spoke departure
2,23816,3410,BTX,EWX,A0XX1J,09-16-22,How satisfied were you with the food & beverag...,5,NaN,5,...,E7F,UAX,266,Short,3,C132,Domestic,member,GOGO,spoke departure
3,43360,6157,ATX,IAX,A0XX2T,09-28-22,How satisfied were you with the food & beverag...,2,NaN,2,...,E7A,UAX,689,Medium,T14,B86,Domestic,member,GOGO,spoke departure
4,21371,1573,DEX,EWX,A0XX2V,09-14-22,How satisfied were you with the food & beverag...,2,NaN,2,...,78P,UA,1605,Medium,B36,C108,Domestic,member,PANASONIC,hub departure


# Data Cleaning and Feature Engineering for Correlation Analysis


In [93]:
#dropping unesccesary columns
lst=['SN','flight_number', 'origin_station_code', 'destination_station_code','question_text',
       'record_locator', 'scheduled_departure_date', 'score','entrée_score',
       'driver_sub_group1', 'driver_sub_group2',
       'cabin_code_desc',
       'seat_factor_band', 'generation', 
       'fleet_usage', 'equipment_type_code',
       'departure_gate', 'arrival_gate', 'media_provider', 'fleet_type_description']

df1=df.drop(columns=lst)

In [107]:
#feature engineering
df1.loc[df['satisfaction_type']=='Satisfied', 'Satisfaction_type']=1
df1.loc[df['satisfaction_type']=='Dissatisfied', 'Satisfaction_type']=0

df1.loc[df['cabin_name']=='Economy', 'cabin_type']=0
df1.loc[df['cabin_name']=='Economy Plus', 'cabin_type']=1


df1.loc[df['international_domestic_indicator']=='Domestic', 'ID_indicator']=0
df1.loc[df['international_domestic_indicator']=='International', 'ID_indicator']=1

df1.loc[df['arrival_delay_group']=='Early & Ontime', 'arrival']=1
df1.loc[df['arrival_delay_group']=='Delayed', 'arrival']=0

df1.loc[df['loyalty_program_level']=='non-elite', 'loyalty']=0
df1.loc[df['loyalty_program_level']=='premier silver', 'loyalty']=1
df1.loc[df['loyalty_program_level']=='premier gold', 'loyalty']=2
df1.loc[df['loyalty_program_level']=='premier platinum', 'loyalty']=3
df1.loc[df['loyalty_program_level']=='premier 1k', 'loyalty']=4
df1.loc[df['loyalty_program_level']=='global services', 'loyalty']=5

df1.loc[df['ua_uax']=='UA', 'flight_type']=0
df1.loc[df['ua_uax']=='UAX', 'flight_type']=1

df1.loc[df['haul_type']=='Short', 'haul_type']=0
df1.loc[df['haul_type']=='Medium', 'haul_type']=1
df1.loc[df['haul_type']=='Long', 'haul_type']=2

df1.loc[df['hub_spoke']=='hub departure', 'Dept_type']=1           #hub departure is more beneficial as stated case background
df1.loc[df['hub_spoke']=='spoke departure', 'Dept_type']=0         

df1.loc[df['response_group']=='member', 'Responsegrp_type']=1
df1.loc[df['response_group']=='non-member', 'Responsegrp_type']=0


In [118]:
df1.head()

,actual_score,satisfaction_type,arrival_delay_minutes,arrival_delay_group,cabin_name,entity,number_of_legs,seat_factor_band1,loyalty_program_level,ua_uax,...,response_group,hub_spoke,Satisfaction_type,cabin_type,ID_indicator,arrival,loyalty,flight_type,Dept_type,Responsegrp_type
0,5,Satisfied,44,Delayed,NaN,Domestic,2,90,premier gold,UA,...,member,hub departure,1.0,NaN,0.0,0.0,2.0,0.0,1.0,1.0
1,5,Satisfied,-19,Early & Ontime,Economy Plus,Domestic,2,90,non-elite,UAX,...,member,spoke departure,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2,5,Satisfied,-34,Early & Ontime,NaN,Domestic,1,90,non-elite,UAX,...,member,spoke departure,1.0,NaN,0.0,1.0,0.0,1.0,0.0,1.0
3,2,Dissatisfied,1,Delayed,Economy,Domestic,1,90,premier 1k,UAX,...,member,spoke departure,0.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0
4,2,Dissatisfied,23,Delayed,Economy Plus,Domestic,2,90,premier silver,UA,...,member,hub departure,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


Correlation Analysis Matrix using the following feature:
1. Satisfaction Scores (actual_score)
2. Arrival Delay (arrival_delay_minutes)
3. Distance Travelled(actual_flown_miles)
4. International and Domestic flight(ID_indicator)
5. Cabin Type(cabin_type)
6. Loyalty program level(loyalty_program_level)
7. No. of legs(number_of_legs)
8. Haul Type(haul_type)
9. Flight type(flight_type)
10. Hub spoke type(Dept_type)
11. Response Group(Responsegrp_type)
12. Seat factor (seat_factor_band1)

In [109]:
cm = sns.light_palette("green", as_cmap=True)
correlation_analysis=pd.DataFrame(df1[['actual_score', 'arrival_delay_minutes',
       'arrival_delay_group', 'number_of_legs',
       'seat_factor_band1', 'loyalty_program_level', 'ua_uax',
       'actual_flown_miles', 'haul_type',
        'hub_spoke', 'Satisfaction_type', 'cabin_type',
       'ID_indicator', 'arrival', 'loyalty', 'flight_type', 'Dept_type',
       'Responsegrp_type']].corr()['Satisfaction_type']).reset_index()
correlation_analysis.columns=['Impacting Variable','Degree of Linear Impact (Correlation)']
correlation_analysis=correlation_analysis[correlation_analysis['Impacting Variable']!='Satisfaction_type']
correlation_analysis=correlation_analysis.sort_values('Degree of Linear Impact (Correlation)',ascending=False)
correlation_analysis.style.background_gradient(cmap=cm).set_precision(2)



C:\Users\vaibh\AppData\Local\Temp\ipykernel_3656\705167941.py:12: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  correlation_analysis.style.background_gradient(cmap=cm).set_precision(2)


,Impacting Variable,Degree of Linear Impact (Correlation)
0,actual_score,0.87
8,arrival,0.05
2,number_of_legs,0.01
11,Dept_type,0.01
12,Responsegrp_type,-0.01
10,flight_type,-0.01
7,ID_indicator,-0.02
3,seat_factor_band1,-0.02
9,loyalty,-0.02
6,cabin_type,-0.03


Sentiment Analysis for Customer Comments

In [103]:
import nltk

In [110]:
dataset=pd.read_csv('Customer comments.csv')

In [111]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()
dataset['polarity scores']= dataset['verbatim_text'].apply(lambda x: sia.polarity_scores(x)['compound'])

Differentiating Sentiments using Polarity Scores

In [116]:
dataset.loc[dataset['polarity scores']<0, 'Sentiment']="Negative"
dataset.loc[dataset['polarity scores']<0.5, 'Sentiment']="Neutral"
dataset.loc[0.5<=dataset['polarity scores'], 'Sentiment']="Positive"
dataset.head()